if the code is running don't touch it . So , don't touch it. 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.cluster import KMeans

# Step 1: Load and preprocess the data
# Load the CSV file
for i in range(1, 28 + 1):
        for group_number in range(1, 4 + 1):
            signal = f"S{i:02}"
            group = f"G{group_number}"
            file_path = f'C:/Users/ASUS/OneDrive/Desktop/GAMEEMO/({signal})/Preprocessed EEG Data/.csv format/final_mean_{signal}.csv'
data = pd.read_csv(file_path)

# Drop unnecessary columns
data = data.dropna(axis=1, how='all')  # Drop completely empty columns

# Ensure we have valid numeric data
if 'Final Mean' in data.columns:
    X = data['Final Mean'].values.reshape(-1, 1)  # Use 'Final Mean' as feature
else:
    raise ValueError("The dataset must include a 'Final Mean' column.")

# Handle missing values (fill NaN with the column mean)
if np.isnan(X).any():
    print("Missing values detected. Imputing missing values...")
    X = np.nan_to_num(X, nan=np.nanmean(X))

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 2: Assign synthetic labels using K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
cluster_labels = kmeans.fit_predict(X_scaled)

# Map clusters to moods (e.g., 0: Sad, 1: Neutral, 2: Happy)
cluster_to_mood = {0: 'Sad', 1: 'Neutral', 2: 'Happy'}
mood_labels = [cluster_to_mood[label] for label in cluster_labels]

# Add mood labels to the dataset
data['Mood'] = mood_labels

# Encode target labels for the model
unique_moods = data['Mood'].unique()
mood_mapping = {mood: idx for idx, mood in enumerate(unique_moods)}
y = data['Mood'].map(mood_mapping)
y_categorical = to_categorical(y)

# Step 3: Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=42)

# Step 4: Define a deep learning model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(unique_moods), activation='softmax')  # Output layer for classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 5: Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Step 6: Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Example predictions
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(y_test, axis=1)

# Map predicted classes back to moods
predicted_moods = [unique_moods[idx] for idx in predicted_classes]
# actual_m


this code will create the moods table in the csv file and also identify the mood.

In [25]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Step 1: Iterate through all the folders and files
for i in range(1, 28 + 1):
    # Generate signal identifier
    signal = f"S{i:02}"
    
    # Define input and output file paths
    input_file = rf'C:\Users\ASUS\OneDrive\Desktop\GAMEEMO\({signal})\Preprocessed EEG Data\.csv format\final_mean_{signal}.csv'
    output_file = rf'C:\Users\ASUS\OneDrive\Desktop\GAMEEMO\({signal})\Preprocessed EEG Data\.csv format\eeg_dataset_with_moods_{signal}.csv'
    
    # Check if the input file exists
    if not os.path.exists(input_file):
        print(f"Input file does not exist: {input_file}")
        continue  # Skip to the next iteration if the file doesn't exist
    
    # Ensure the directory for the output file exists
    output_dir = os.path.dirname(output_file)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Step 2: Load and preprocess the data
    try:
        # Load the CSV file
        data = pd.read_csv(input_file)
        
        # Drop unnecessary or completely empty columns
        data = data.dropna(axis=1, how='all')  # Drop completely empty columns
        
        # Ensure we have valid numeric data from the 'Final Mean' column
        if 'Final Mean' in data.columns:
            X = data['Final Mean'].values.reshape(-1, 1)  # Use 'Final Mean' as feature
        else:
            raise ValueError(f"'Final Mean' column missing in {input_file}")
        
        # Handle missing values (fill NaN with the column mean)
        if np.isnan(X).any():
            print(f"Missing values detected in {input_file}. Imputing missing values...")
            X = np.nan_to_num(X, nan=np.nanmean(X))
        
        # Scale features for clustering
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Step 3: Perform K-Means clustering
        kmeans = KMeans(n_clusters=3, random_state=42)
        cluster_labels = kmeans.fit_predict(X_scaled)

        # Step 4: Map clusters to moods
        # Determine the cluster ordering based on the mean of the 'Final Mean' values
        data['Cluster'] = cluster_labels
        cluster_means = data.groupby('Cluster')['Final Mean'].mean().sort_values()
        ordered_clusters = {cluster: i for i, cluster in enumerate(cluster_means.index)}

        # Map clusters to moods (e.g., 0 -> Sad, 1 -> Neutral, 2 -> Happy)
        moods = ['Sad', 'Neutral', 'Happy']  # Adjust mood labels as needed
        cluster_to_mood = {cluster: moods[ordered_clusters[cluster]] for cluster in ordered_clusters}
        data['Mood'] = data['Cluster'].map(cluster_to_mood)

        # Step 5: Save the labeled dataset to a new CSV file
        data.to_csv(output_file, index=False)
        print(f"Labeled dataset saved to {output_file}")

    except Exception as e:
        print(f"Error processing file {input_file}: {e}")


Missing values detected in C:\Users\ASUS\OneDrive\Desktop\GAMEEMO\(S01)\Preprocessed EEG Data\.csv format\final_mean_S01.csv. Imputing missing values...
Labeled dataset saved to C:\Users\ASUS\OneDrive\Desktop\GAMEEMO\(S01)\Preprocessed EEG Data\.csv format\eeg_dataset_with_moods_S01.csv
Missing values detected in C:\Users\ASUS\OneDrive\Desktop\GAMEEMO\(S02)\Preprocessed EEG Data\.csv format\final_mean_S02.csv. Imputing missing values...
Labeled dataset saved to C:\Users\ASUS\OneDrive\Desktop\GAMEEMO\(S02)\Preprocessed EEG Data\.csv format\eeg_dataset_with_moods_S02.csv
Missing values detected in C:\Users\ASUS\OneDrive\Desktop\GAMEEMO\(S03)\Preprocessed EEG Data\.csv format\final_mean_S03.csv. Imputing missing values...
Labeled dataset saved to C:\Users\ASUS\OneDrive\Desktop\GAMEEMO\(S03)\Preprocessed EEG Data\.csv format\eeg_dataset_with_moods_S03.csv
Missing values detected in C:\Users\ASUS\OneDrive\Desktop\GAMEEMO\(S04)\Preprocessed EEG Data\.csv format\final_mean_S04.csv. Imputing m

new code.
iske liye hi mehnat karni hai bas ....


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Step 1: Load and preprocess the data
# Load the CSV file
for i in range(1, 28 + 1):
        for group_number in range(1, 4 + 1):
            signal = f"S{i:02}"
            group = f"G{group_number}"
            file_path = f'C:/Users/ASUS/OneDrive/Desktop/GAMEEMO/({signal})/Preprocessed EEG Data/.csv format/final_mean_{signal}.csv'
data = pd.read_csv(file_path)

# Drop unnecessary columns
data = data.dropna(axis=1, how='all')  # Drop empty columns

# Extract features and target (assuming a target column exists for moods)
# Example: 'Mood' column contains labels like 'Happy', 'Sad', 'Neutral'
if 'Mood' in data.columns:
    X = data.drop(['Mood'], axis=1).select_dtypes(include=[np.number])
    y = data['Mood']
else:
    raise ValueError("The dataset must include a 'Mood' column with labels.")

# Encode target labels
unique_moods = y.unique()
mood_mapping = {mood: idx for idx, mood in enumerate(unique_moods)}
y = y.map(mood_mapping)
y_categorical = to_categorical(y)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 2: Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=42)

# Step 3: Define a deep learning model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(unique_moods), activation='softmax')  # Output layer for classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 4: Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Step 5: Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Example predictions
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(y_test, axis=1)

# Map predicted classes back to moods
predicted_moods = [unique_moods[idx] for idx in predicted_classes]
actual_moods = [unique_moods[idx] for idx in actual_classes]

# Display example results
for i in range(5):
    print(f"Actual Mood: {actual_moods[i]}, Predicted Mood: {predicted_moods[i]}")


mam ka code 

In [ ]:
# Step 1: Load and Preprocess EEG Data
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# Load EEG dataset
try:
    for i in range(1, 28 + 1):
        for group_number in range(1, 4 + 1):
            signal = f"S{i:02}"
            group = f"G{group_number}"
            file_path = f'C:/Users/ASUS/OneDrive/Desktop/GAMEEMO/({signal})/Preprocessed EEG Data/.csv format/final_mean_{signal}.csv'
    data = pd.read_csv(file_path)  # Replace with your dataset path
except FileNotFoundError:
    print(f"Error: File {file_path} not found. Please provide the correct path.")
    exit()

# Ensure dataset has no missing values
if data.isnull().sum().sum() > 0:
    print("Error: Dataset contains missing values. Please handle them before proceeding.")
    exit()

# Ensure dataset is numeric
if not np.issubdtype(data.dtypes.values[0], np.number):
    print("Error: Non-numeric data detected. Ensure all features are numeric.")
    exit()

# Scale features
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Optional: Reduce dimensions for visualization
pca = PCA(n_components=2)  # For 2D visualization
data_pca = pca.fit_transform(data_scaled)

# Visualize data distribution
plt.figure(figsize=(8, 6))
plt.scatter(data_pca[:, 0], data_pca[:, 1], alpha=0.7)
plt.title('EEG Data Distribution (PCA)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

# Step 2: Cluster Data into 3 Groups using K-Means Clustering

# Cluster into 3 groups
kmeans = KMeans(n_clusters=3, random_state=42)
labels = kmeans.fit_predict(data_scaled)

# Add cluster labels to the dataset
data['cluster_labels'] = labels

# Visualize clustering
plt.figure(figsize=(8, 6))
plt.scatter(data_pca[:, 0], data_pca[:, 1], c=labels, cmap='viridis', alpha=0.7)
plt.title('Clustering Results')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(label='Cluster')
plt.show()

# Step 3: Map Clusters to Desired Labels
# Analyze the clusters and assign labels based on heuristics or domain knowledge.

# Example heuristic for mapping
# Assuming mean values of features determine emotional state:
# Cluster with the lowest mean -> -1 (Sad)
# Cluster with middle mean ->  0 (Neutral)
# Cluster with highest mean ->  1 (Happy)
cluster_means = data.drop(['cluster_labels'], axis=1).groupby(data['cluster_labels']).mean()
cluster_map = cluster_means.mean(axis=1).sort_values().index

# Map clusters to -1, 0, 1
cluster_to_emotion = {cluster_map[0]: -1, cluster_map[1]: 0, cluster_map[2]: 1}
data['emotion_labels'] = data['cluster_labels'].map(cluster_to_emotion)

print("Cluster to Emotion Mapping:", cluster_to_emotion)
print(data[['cluster_labels', 'emotion_labels']].head())

# Step 4: Train a Model Using New Labels
# Separate features and labels
X = data.drop(['cluster_labels', 'emotion_labels'], axis=1)
y = data['emotion_labels']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
